# Test 3DVar and 4DVar implementation

In [1]:
import torch
from deepda import apply_3DVar, apply_4DVar, forwardModel_r
from math import ceil

In [2]:
def H(x: torch.Tensor):
    return x

In [3]:
B = torch.eye(3, device="cuda")
R = torch.eye(3, device="cuda")
y = torch.tensor([10., 20., 30.], device="cuda")
xb = torch.zeros_like(y, device="cuda")

In [4]:
apply_3DVar(H, B, R, xb, y, learning_rate=2)

Iterations: 0, Norm of J gradient: 74.83314514160156
Iterations: 1, Norm of J gradient: 62.22539520263672
Iterations: 2, Norm of J gradient: 50.396549224853516
Iterations: 3, Norm of J gradient: 39.807682037353516
Iterations: 4, Norm of J gradient: 30.810375213623047
Iterations: 5, Norm of J gradient: 23.481937408447266
Iterations: 6, Norm of J gradient: 18.005842208862305
Iterations: 7, Norm of J gradient: 14.986854553222656
Iterations: 8, Norm of J gradient: 14.758964538574219
Iterations: 9, Norm of J gradient: 16.46552276611328
Iterations: 10, Norm of J gradient: 18.841018676757812
Iterations: 11, Norm of J gradient: 21.076297760009766
Iterations: 12, Norm of J gradient: 22.750995635986328
Iterations: 13, Norm of J gradient: 23.645172119140625
Iterations: 14, Norm of J gradient: 23.67448616027832
Iterations: 15, Norm of J gradient: 22.87781524658203
Iterations: 16, Norm of J gradient: 21.400211334228516
Iterations: 17, Norm of J gradient: 19.463558197021484
Iterations: 18, Norm of J

tensor([ 5.0000, 10.0000, 15.0000], device='cuda:0')

In [5]:
# We define the control parameters here
rayleigh = 35
prandtl = 10.
b = 8./3.
# rayleigh = 0.
# prandtl = 0.
# b = 0.
# initial condition for the true reference trajectory
x0 = torch.tensor([0., 1., 2.], device="cuda")

# integration time parameter
dt = 1.e-3      # This is time step size
T = 2.         # Total integration time, can be as short as 10 to speed things up
n_steps = ceil(T / dt)
time = torch.linspace(0., T, n_steps + 1, device="cuda")  # array of discrete times

# numerical integration given initial conditions and control parameters
xt = forwardModel_r(x0, time, rayleigh, prandtl, b)

In [6]:
# Which variables do we observe?
WhichVariablesAreObserved = torch.tensor([1, 1, 1])
#  Determines which variables are available to
#  the EnKF. For example:
#  WhichVariablesAreObserved = [1 1 1];
#  means: X, Y, Z are observed
#  WhichVariablesAreObserved = [1 0 1];
#  means: X and Z are observed
#  WhichVariablesAreObserved = [1 0 0];
# means: X is observed
sigobs = 2.  # standard deviation of the observation noise
# We generate the synthetic data
#  Construct observation matrix H
#  ........................................................................
# H = torch.diag(WhichVariablesAreObserved)
y_size = int(WhichVariablesAreObserved.sum())
# How often do we observe the true state?
dtobs = 0.5  # time between observations
nobs = ceil(T / dtobs) - 1  # number of times observations are performed
# no observation at t=0
gap = int(dtobs / dt)  # number of time steps between each observation
time_obs = time[gap::gap]
# Generate vector of observations
y = torch.zeros((y_size, nobs), device="cuda")
R = torch.diag(torch.tile(torch.tensor(sigobs**2, device="cuda"), (y_size,)))
sqrt_s = torch.sqrt(R)
# y = Hxt
y = H(xt[:, gap::gap])
# compute observation error
noise = sqrt_s @ torch.randn(size=y.shape, device="cuda")
# y = Hxt + epsilon
y = y + noise

In [7]:
apply_4DVar(nobs, time_obs, gap, forwardModel_r, H, B, R, x0, y, model_args=(rayleigh, prandtl, b), learning_rate=0.0125)

Iterations: 0, Norm of J gradient: 1959.23974609375
Iterations: 1, Norm of J gradient: 1789.276123046875
Iterations: 2, Norm of J gradient: 1635.7635498046875
Iterations: 3, Norm of J gradient: 1497.1085205078125
Iterations: 4, Norm of J gradient: 1371.8905029296875
Iterations: 5, Norm of J gradient: 1258.741943359375
Iterations: 6, Norm of J gradient: 1156.4400634765625
Iterations: 7, Norm of J gradient: 1063.895263671875
Iterations: 8, Norm of J gradient: 980.1072998046875
Iterations: 9, Norm of J gradient: 904.165283203125
Iterations: 10, Norm of J gradient: 835.3040161132812
Iterations: 11, Norm of J gradient: 772.7566528320312
Iterations: 12, Norm of J gradient: 715.8984985351562
Iterations: 13, Norm of J gradient: 664.1416625976562
Iterations: 14, Norm of J gradient: 616.9940185546875
Iterations: 15, Norm of J gradient: 573.9701538085938
Iterations: 16, Norm of J gradient: 534.6973266601562
Iterations: 17, Norm of J gradient: 498.7599182128906
Iterations: 18, Norm of J gradient: 

tensor([ 0.1351,  1.2994, -5.3740], device='cuda:0')